In [1]:
import numpy as np
from ortools.sat.python import cp_model

## Data functions

In [2]:
def load_input_data(input_data):
    """
    Return input data as numpy array of [edge 1, edge 2]
    """

    # parse the input
    lines = input_data.split('\n')

    firstLine = lines[0].split()
    num_nodes = int(firstLine[0])
    num_edges = int(firstLine[1])

    out_array = np.zeros((num_edges, 2))

    for i in range(num_edges):
        line = lines[i + 1]
        parts = line.split()
        out_array[i] = np.array([int(parts[0]), int(parts[1])])

    return out_array, num_nodes

In [225]:
input_file = open('data/gc_50_3','r').read()

In [226]:
edge_array, num_nodes = load_input_data(input_file)
edge_array

array([[ 0.,  3.],
       [ 0.,  5.],
       [ 0.,  8.],
       [ 0., 10.],
       [ 0., 42.],
       [ 0., 43.],
       [ 0., 45.],
       [ 0., 49.],
       [ 1.,  5.],
       [ 1., 13.],
       [ 1., 15.],
       [ 1., 16.],
       [ 1., 20.],
       [ 1., 22.],
       [ 1., 24.],
       [ 1., 25.],
       [ 1., 27.],
       [ 1., 31.],
       [ 1., 35.],
       [ 1., 39.],
       [ 1., 45.],
       [ 1., 46.],
       [ 1., 47.],
       [ 2.,  3.],
       [ 2.,  9.],
       [ 2., 10.],
       [ 2., 12.],
       [ 2., 15.],
       [ 2., 21.],
       [ 2., 28.],
       [ 2., 36.],
       [ 2., 42.],
       [ 2., 46.],
       [ 2., 47.],
       [ 2., 48.],
       [ 3.,  5.],
       [ 3.,  7.],
       [ 3.,  8.],
       [ 3.,  9.],
       [ 3., 11.],
       [ 3., 25.],
       [ 3., 30.],
       [ 3., 33.],
       [ 3., 35.],
       [ 3., 36.],
       [ 3., 40.],
       [ 3., 45.],
       [ 4., 12.],
       [ 4., 15.],
       [ 4., 16.],
       [ 4., 20.],
       [ 4., 24.],
       [ 4.,

In [215]:
def order_nodes_by_order_desc(edge_array):
    
    node_order_dict = {}
    
    for node_index in range(int(edge_array.max())):
        count_node_edges = (edge_array == node_index).sum()
        node_order_dict[node_index] = count_node_edges
        
    node_by_order_desc = [tup[0] for tup in sorted(node_order_dict.items(), key=lambda x: x[1], reverse=True)]
        
    return node_by_order_desc

In [227]:
d = order_nodes_by_order_desc(edge_array)
d

[45,
 7,
 33,
 35,
 32,
 47,
 16,
 27,
 42,
 44,
 6,
 11,
 15,
 19,
 20,
 29,
 31,
 43,
 1,
 9,
 22,
 3,
 8,
 12,
 26,
 28,
 4,
 5,
 17,
 37,
 41,
 46,
 2,
 10,
 14,
 24,
 34,
 48,
 23,
 25,
 30,
 36,
 40,
 13,
 18,
 38,
 39,
 0,
 21]

In [254]:
def prepare_output_data(solution_dict, is_provably_optimal=False):
    """
    Return output in specified format.
    """

    if is_provably_optimal:
        optimal = str(1)
    else:
        optimal = str(0)

    output_data = str(solution_dict['num_colours']) + ' ' + optimal + '\n'
    output_data += ' '.join(map(str, solution_dict['solution_array']))

    return output_data

In [255]:
prepare_output_data(get_solution_dict(nc_var, solv, num_nodes))

'9 0\n5 1 5 6 4 3 7 1 4 0 2 1 7 2 5 3 0 3 5 7 0 0 4 4 3 4 3 0 2 1 1 0 2 4 0 3 4 4 3 0 5 5 1 6 2 8 2 3 1 0'

# fuck it just use pyomo cos ortools is v hard to use

## CP model

In [52]:
def create_node_colour_variables(model, edge_array, num_nodes):
    
    # fine for now
    num_colours = num_nodes 
    
    node_colour_variables = {}
    for node_index in range(num_nodes):
        node_colour_variables[node_index] = model.NewIntVar(0, num_colours-1, 'node_%i' % node_index)
    
    return node_colour_variables, num_colours

In [53]:
def create_node_colour_constraints(model, edge_array, node_colour_variables):
    
    for edge_index in range(edge_array.shape[0]):
        model.Add(node_colour_variables[edge_array[edge_index, 0]] != node_colour_variables[edge_array[edge_index, 1]])
        
    return model

In [404]:
def create_node_colour_bool_variables(model, edge_array, num_nodes):
    
    # fine for now
    num_colours = num_nodes 
    
    node_colour_bool_variables = {}
    for node_index in range(num_nodes):
        for colour_index in range(num_colours):
            node_colour_bool_variables[(node_index, colour_index)] = \
                model.NewBoolVar('node_%i_colour_%i' % (node_index, colour_index))
    
    return node_colour_bool_variables

In [416]:
def create_node_colour_bool_constraints(model, edge_array, node_colour_bool_variables, node_colour_variables, num_colours):
    
    for node_index in range(num_nodes):
        for colour_index in range(num_colours):
            model.Add(node_colour_variables[node_index] == colour_index).\
                OnlyEnforceIf(node_colour_bool_variables[(node_index, colour_index)])
            model.Add(node_colour_variables[node_index] != colour_index).\
                OnlyEnforceIf(node_colour_bool_variables[(node_index, colour_index)].Not())
        
    return model

In [ ]:
#def create_node_colour_bool_constraints(model, edge_array, node_colour_bool_variables, node_colour_variables):
    
    for node_index in range(num_nodes):
        for colour_index in range(num_colours):
            model.Add(node_colour_bool_variables[(node_index, colour_index)] == node_colour_variables[node_index])
    
    #for edge_index in range(edge_array.shape[0]):
    #    model.Add(node_colour_variables[edge_array[edge_index, 0]] != node_colour_variables[edge_array[edge_index, 1]])
        
    return model

In [349]:
def create_colour_used_variables(model, num_colours):
    
    colour_used_variables = {}
    for colour_index in range(num_colours):
        colour_used_variables[colour_index] = model.NewBoolVar('colour_%i' % colour_index)
        
    return colour_used_variables

In [348]:
#def create_colour_used_variables(model, num_colours, num_nodes):
    
#    colour_used_variables = {}
#    for colour_index in range(num_colours):
#        colour_used_variables[colour_index] = model.NewIntVar(0, num_nodes-1, 'colour_%i' % colour_index)
#        
#    return colour_used_variables

In [69]:
#def create_colour_used_constraints(model, node_colour_variables, colour_used_variables, num_nodes, num_colours):
    
    for colour_index in range(num_colours):
        model.Add(colour_used_variables[colour_index] == model.Or([node_colour_variables[node_index] == colour_index 
                                                                          for node_index in node_colour_variables.keys()]))
        
    return model

In [316]:
def create_colour_used_constraints(model, node_colour_variables, colour_used_variables, num_nodes, num_colours):
    
    for colour_index in range(num_colours):
        model.Add(colour_used_variables[colour_index] == min(1,sum([1 for node_index in node_colour_variables.keys()
                                                          if node_colour_variables[node_index] == colour_index])))
        
    return model

In [397]:
def create_colour_used_constraints(model, node_colour_variables, colour_used_variables, num_nodes, num_colours):
    
    for colour_index, colour_used in colour_used_variables.items():
        for node_index, node_colour in node_colour_variables.items():
            if node_colour == colour_index:
                #print(node_colour)
                colour_used = True
            else:
                colour_used = False
        model.Add(colour_used_variables[colour_index] == colour_used)
        
    return model

In [64]:
#def create_colour_used_constraints(model, node_colour_variables, colour_used_variables, num_nodes, num_colours):
    
    for colour_index in range(num_colours):
        for node_index in range(num_nodes):
            model.Add(colour_used_variables[colour_index] == True).OnlyEnforceIf(node_colour_variables[node_index] == colour_index)
        
    return model

In [217]:
#def create_colour_symmetry_breaking_constraints(model, edge_array, node_colour_variables, num_nodes):
    
    node_order = order_nodes_by_order_desc(edge_array)
    
    model.Add(node_colour_variables[node_order[0]] == 0)
    for counter, node_index in enumerate(node_order[1:]):
        model.Add(node_colour_variables[node_index] <= max([node_colour_variables[node_index_2] 
                                                            for node_index_2 in node_order[:counter+1]]) + 1)
        
    return model


In [318]:
def create_colour_symmetry_breaking_constraints(model, colour_used_variables, num_colours):
    
    for colour_index in range(num_colours-1):
        model.Add(colour_used_variables[colour_index] <= colour_used_variables[colour_index+1])
        
    return model


In [320]:
def create_objective(model, node_colour_variables, num_nodes):
    
    # create objective value variable
    obj_val_var = model.NewIntVar(0, num_nodes-1, 'num_distinct_colours')
    
    # add constraint to set the variable
    model.Add(obj_val_var == max([node_colour_variables[node_index] for node_index in range(num_nodes)]))
    
    model.Minimize(obj_val_var)
    
    return model, obj_val_var
              

In [413]:
def create_model(edge_array, num_nodes):
    
    # create model
    model = cp_model.CpModel()
    #solver = cp_model.CpSolver()
    
    # create variables
    node_colour_bool_variables = create_node_colour_bool_variables(model, edge_array, num_nodes)
    node_colour_variables, num_colours = create_node_colour_variables(model, edge_array, num_nodes)
    #colour_used_variables = create_colour_used_variables(model, num_colours)
    #colour_used_variables = create_colour_used_variables(model, num_colours, num_nodes)
    
    
    # add constraints
    model = create_node_colour_bool_constraints(model, edge_array, node_colour_bool_variables, 
                                                node_colour_variables, num_colours)
    model = create_node_colour_constraints(model, edge_array, node_colour_variables)
    #model = create_colour_used_constraints(model, node_colour_variables, colour_used_variables, num_nodes, num_colours)
    #model = create_colour_used_constraints(model, solver, node_colour_variables, colour_used_variables, num_nodes, num_colours)
    #model = create_colour_symmetry_breaking_constraints(model, edge_array, node_colour_variables, num_nodes)
    #model = create_colour_symmetry_breaking_constraints(model, colour_used_variables, num_colours)
    
    
    # add objective
    #model, obj_val_variable = create_objective(model, node_colour_variables, num_nodes)
    #model.Add(obj_val_variable <= 8)
    
    return model, node_colour_variables#, colour_used_variables, obj_val_variable

In [417]:
#mod, nc_var, cu_var, obj_val_var = create_model(edge_array, num_nodes)
mod, nc_var = create_model(edge_array, num_nodes)

KeyError: 0

In [379]:
def solve_model(model, node_colour_variables):
    
    # solve model
    solver = cp_model.CpSolver()
    status = solver.Solve(model)
    
    return model, node_colour_variables, solver, status

In [392]:
mod, nc_var, solv, stat = solve_model(mod,  nc_var)

In [362]:
solv.Value(obj_val_var)

0

In [280]:
def print_results2(model, cu_variables, solver):
    
    #max_obj = 0
    
    for colour_index, colour_var in cu_variables.items():
        val = solver.Value(colour_var)
        #max_obj = max(max_obj, val)
        print('Node ' + str(colour_index) + ': ' + str(val))
        
    #print('Objective: ' + str(max_obj+1))

In [355]:
nc_var.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])

In [334]:
[node_colour for node_index, node_colour in nc_var.items() if solv.Value(node_colour) == 10]#[0] == 9

[]

In [393]:
print_results2(mod, cu_var, solv)

Node 0: 1
Node 1: 1
Node 2: 1
Node 3: 1
Node 4: 1
Node 5: 1
Node 6: 1
Node 7: 1
Node 8: 1
Node 9: 1
Node 10: 1
Node 11: 1
Node 12: 1
Node 13: 1
Node 14: 1
Node 15: 1
Node 16: 1
Node 17: 1
Node 18: 1
Node 19: 1
Node 20: 1
Node 21: 1
Node 22: 1
Node 23: 1
Node 24: 1
Node 25: 1
Node 26: 1
Node 27: 1
Node 28: 1
Node 29: 1
Node 30: 1
Node 31: 1
Node 32: 1
Node 33: 1
Node 34: 1
Node 35: 1
Node 36: 1
Node 37: 1
Node 38: 1
Node 39: 1
Node 40: 1
Node 41: 1
Node 42: 1
Node 43: 1
Node 44: 1
Node 45: 1
Node 46: 1
Node 47: 1
Node 48: 1
Node 49: 1


In [272]:
if stat == cp_model.OPTIMAL:
    print('y')

y


In [170]:
def print_results(model, variables, solver):
    
    max_obj = 0
    
    for node_index, node_var in variables.items():
        val = solver.Value(node_var)
        max_obj = max(max_obj, val)
        print('Node ' + str(node_index) + ': ' + str(val))
        
    print('Objective: ' + str(max_obj+1))

In [394]:
print_results(mod, nc_var, solv)

Node 0: 5
Node 1: 1
Node 2: 5
Node 3: 6
Node 4: 4
Node 5: 3
Node 6: 7
Node 7: 1
Node 8: 4
Node 9: 0
Node 10: 2
Node 11: 1
Node 12: 7
Node 13: 2
Node 14: 5
Node 15: 3
Node 16: 0
Node 17: 3
Node 18: 5
Node 19: 7
Node 20: 0
Node 21: 0
Node 22: 4
Node 23: 4
Node 24: 3
Node 25: 4
Node 26: 3
Node 27: 0
Node 28: 2
Node 29: 1
Node 30: 1
Node 31: 0
Node 32: 2
Node 33: 4
Node 34: 0
Node 35: 3
Node 36: 4
Node 37: 4
Node 38: 3
Node 39: 0
Node 40: 5
Node 41: 5
Node 42: 1
Node 43: 6
Node 44: 2
Node 45: 8
Node 46: 2
Node 47: 3
Node 48: 1
Node 49: 0
Objective: 9


In [258]:
edge_array

array([[ 0.,  3.],
       [ 0.,  5.],
       [ 0.,  8.],
       [ 0., 10.],
       [ 0., 42.],
       [ 0., 43.],
       [ 0., 45.],
       [ 0., 49.],
       [ 1.,  5.],
       [ 1., 13.],
       [ 1., 15.],
       [ 1., 16.],
       [ 1., 20.],
       [ 1., 22.],
       [ 1., 24.],
       [ 1., 25.],
       [ 1., 27.],
       [ 1., 31.],
       [ 1., 35.],
       [ 1., 39.],
       [ 1., 45.],
       [ 1., 46.],
       [ 1., 47.],
       [ 2.,  3.],
       [ 2.,  9.],
       [ 2., 10.],
       [ 2., 12.],
       [ 2., 15.],
       [ 2., 21.],
       [ 2., 28.],
       [ 2., 36.],
       [ 2., 42.],
       [ 2., 46.],
       [ 2., 47.],
       [ 2., 48.],
       [ 3.,  5.],
       [ 3.,  7.],
       [ 3.,  8.],
       [ 3.,  9.],
       [ 3., 11.],
       [ 3., 25.],
       [ 3., 30.],
       [ 3., 33.],
       [ 3., 35.],
       [ 3., 36.],
       [ 3., 40.],
       [ 3., 45.],
       [ 4., 12.],
       [ 4., 15.],
       [ 4., 16.],
       [ 4., 20.],
       [ 4., 24.],
       [ 4.,

In [252]:
def get_solution_dict(variables, solver, num_nodes):
    
    max_obj = 0
    
    solution_array = np.zeros(num_nodes)
    
    for node_index, node_var in variables.items():
        val = solver.Value(node_var)
        solution_array[node_index] = val
        max_obj = max(max_obj, val)

    solution_dict = {
        'solution_array': solution_array.astype(int),
        'num_colours': max_obj+1
    }
        
    return solution_dict

In [253]:
get_solution_dict(nc_var, solv, num_nodes)

{'solution_array': array([5, 1, 5, 6, 4, 3, 7, 1, 4, 0, 2, 1, 7, 2, 5, 3, 0, 3, 5, 7, 0, 0,
        4, 4, 3, 4, 3, 0, 2, 1, 1, 0, 2, 4, 0, 3, 4, 4, 3, 0, 5, 5, 1, 6,
        2, 8, 2, 3, 1, 0]), 'num_colours': 9}

In [256]:
def solve_it_cp(edge_array, num_nodes):
    
    model, nc_vars = create_model(edge_array, num_nodes)
    
    model, nc_vars, solv, stat = solve_model(model, nc_vars)
    
    solution_dict = get_solution_dict(nc_vars, solv, num_nodes)
    
    return solution_dict

In [257]:
solve_it_cp(edge_array, num_nodes)

{'solution_array': array([5, 1, 5, 6, 4, 3, 7, 1, 4, 0, 2, 1, 7, 2, 5, 3, 0, 3, 5, 7, 0, 0,
        4, 4, 3, 4, 3, 0, 2, 1, 1, 0, 2, 4, 0, 3, 4, 4, 3, 0, 5, 5, 1, 6,
        2, 8, 2, 3, 1, 0]), 'num_colours': 9}

In [403]:
def create_node_colour_variables(model, edge_array, num_nodes):
    
    # fine for now
    num_colours = num_nodes 
    
    node_colour_variables = {}
    for node_index in range(num_nodes):
        for colour_index in range(num_colours):
            node_colour_variables[(node_index, colour_index)] = \
                model.NewBoolVar('node_%i_colour_%i' % (node_index, colour_index))
    
    return node_colour_variables, num_colours

In [ ]:
def create_node_colour_constraints(model, edge_array, node_colour_variables):
    
    for edge_index in range(edge_array.shape[0]):
        model.Add(node_colour_variables[edge_array[edge_index, 0]] != node_colour_variables[edge_array[edge_index, 1]])
        
    return model